In [13]:
import pandas as pd 
import numpy as np 
import os
import datetime as dt

## [Hutchinson Drought Index]

In [14]:
data = pd.read_csv('raw/australia/drought_index.csv',                    
                   skiprows=1, usecols=[1, 2], 
                   parse_dates=True,
                   index_col = 0,
                   names=['Date', 'Rain'])
data.head()

,Rain
Date,
1892-01-31,2.5
1892-02-29,12.2
1892-03-31,1.8
1892-04-30,1.5
1892-05-31,16.8


In [15]:
data['Rain_6'] = data['Rain'].rolling(6).sum()
df_6mon = data[['Rain_6']].dropna()
droughtThreshold = 0.375
df_6mon['HDI'] = np.nan

for imon in np.arange(1, 13):
    sinds = df_6mon.index.month==imon
    
    x = df_6mon[sinds]       
    y = (x.rank()-1.0)/(len(x)-1.0)
    z = 8.0*(y-0.5)    
    df_6mon.loc[sinds, 'HDI'] = z.values[:, 0]

data['HDI'] = df_6mon['HDI']
del df_6mon
data.head(7)

,Rain,Rain_6,HDI
Date,,,
1892-01-31,2.5,NaN,NaN
1892-02-29,12.2,NaN,NaN
1892-03-31,1.8,NaN,NaN
1892-04-30,1.5,NaN,NaN
1892-05-31,16.8,NaN,NaN
1892-06-30,1.0,35.8,-3.394958
1892-07-31,11.7,45.0,-2.453782


In [17]:
data.reset_index(inplace=True)
data['Year'] = data['Date'].dt.year
data_final = data.groupby('Year').mean()
data_final.reset_index(inplace=True)
data_final = data_final[['Year','HDI']]
data_final.head()

,Year,HDI
0,1892,-0.350540
1,1893,0.881451
2,1894,1.646726
3,1895,0.737027
4,1896,-1.661112


In [18]:
dfs = []
for f in os.listdir('raw/australia'):
    print(f)
    if f in ["hail.csv",]:
        df = pd.read_csv(f'raw/australia/{f}',usecols=np.arange(1,7),error_bad_lines=False)
    elif f == "waterspout.csv":
        df = pd.read_csv(f'raw/australia/{f}',usecols=np.arange(1,6),error_bad_lines=False)
    elif f == "rain.csv":
        df = pd.read_csv(f'raw/australia/{f}',usecols=np.arange(1,8),error_bad_lines=False)
    elif f == "ligthning.csv":
        df = pd.read_csv(f'raw/australia/{f}',usecols=np.arange(1,11),error_bad_lines=False)
    elif f == "tornado.csv":
        df = pd.read_csv(f'raw/australia/{f}',usecols=np.arange(1,9),error_bad_lines=False)
    elif f == "wind.csv":
        df = pd.read_csv(f'raw/australia/{f}',usecols=np.arange(1,10),error_bad_lines=False)
    df = pd.read_csv(f'raw/australia/{f}',error_bad_lines=False)
    dfs.append(df)

hail.csv
rain.csv
australia_max_temp.csv
australia_rainfall_anomaly.csv
australia_min_temp.csv
australia_cloud.csv
avg_temp.csv
tornado.csv
lightning.csv
wind.csv
b'Skipping line 13: expected 9 fields, saw 10\nSkipping line 14: expected 9 fields, saw 11\nSkipping line 51: expected 9 fields, saw 10\nSkipping line 71: expected 9 fields, saw 12\nSkipping line 100: expected 9 fields, saw 10\nSkipping line 101: expected 9 fields, saw 11\nSkipping line 107: expected 9 fields, saw 11\nSkipping line 114: expected 9 fields, saw 10\nSkipping line 120: expected 9 fields, saw 10\nSkipping line 129: expected 9 fields, saw 11\nSkipping line 131: expected 9 fields, saw 10\nSkipping line 139: expected 9 fields, saw 11\nSkipping line 140: expected 9 fields, saw 11\nSkipping line 141: expected 9 fields, saw 10\nSkipping line 147: expected 9 fields, saw 12\nSkipping line 159: expected 9 fields, saw 11\nSkipping line 160: expected 9 fields, saw 13\nSkipping line 163: expected 9 fields, saw 10\nSkipping li

In [20]:
hail_df = dfs[0]
rain_df = dfs[1]
tornado_df = dfs[7]
lightning_df = dfs[8]
wind_df = dfs[9]
waterspout_df = dfs[12]
dffs = [dfs[6],dfs[2],dfs[3],dfs[4],dfs[5],dfs[11]]
for d in dffs[1:]:
    d['Year']=''
    yy = d.iloc[:,0]
    for i,n in yy.iteritems():
        n = str(n)
        count = 0
        year = []
        for a in n:
            if count < 4:
                year.append(a)
            count +=1
        year_final = ''.join(year)
        d['Year'][i] = str(year_final)

In [21]:
final_df = pd.DataFrame({'Year':[int(i) for i in range(1900,2021)]})
for d in dffs:
    d['Year'] = d['Year'].astype(int)
    final_df = final_df.merge(d,on="Year",how="left")
final_df = final_df[['Year', 'anomalie','max_temp','min_temp','rainfall_anom',
  'rainfall',' day_cloud']].\
    rename({' day_cloud': 'day_cloud'},axis=1).groupby('Year').mean()
final_df.head()

,anomalie,max_temp,min_temp,rainfall_anom,rainfall,day_cloud
Year,,,,,,
1900,NaN,NaN,NaN,-90.04,376.00,NaN
1901,NaN,NaN,NaN,-61.12,404.92,NaN
1902,NaN,NaN,NaN,-146.51,319.52,NaN
1903,NaN,NaN,NaN,54.24,520.28,NaN
1904,NaN,NaN,NaN,39.79,505.83,NaN


In [22]:
final_df = final_df.merge(data_final,on="Year",how="left")
final_df = final_df.rename({'HDI':'drough_index'},axis=1)
final_df.head()

,Year,anomalie,max_temp,min_temp,rainfall_anom,rainfall,day_cloud,drough_index
0,1900,NaN,NaN,NaN,-90.04,376.00,NaN,-1.446707
1,1901,NaN,NaN,NaN,-61.12,404.92,NaN,-1.622893
2,1902,NaN,NaN,NaN,-146.51,319.52,NaN,-2.833879
3,1903,NaN,NaN,NaN,54.24,520.28,NaN,0.962636
4,1904,NaN,NaN,NaN,39.79,505.83,NaN,0.743199


In [23]:
dfs_d = [hail_df,rain_df,tornado_df,lightning_df,wind_df,waterspout_df]
for df in dfs_d:
    df['date'] = pd.to_datetime(df['Date/Time'])
    df['Year'] = df['date'].dt.year

In [24]:
hail_df = hail_df[['Year', 'Hail size', 'Latitude', 'Longitude', 'Nearest town',
       'State']]
rain_df = rain_df[['Year','Intense precipitation amount', 'Intense precipitation period',
       'Total precipitation', 'Latitude', 'Longitude','State']]
lightning_df = lightning_df[['Year','Flash rate','Latitude', 'Longitude','State','Nearest town']]
tornado_df = tornado_df[['Year','Max speed (km/h)','Latitude', 'Longitude','State','Nearest town']]
wind_df = wind_df[['Year','Max Mean Wind speed','Latitude', 'Longitude','State','Nearest town']]
waterspout_df = waterspout_df[['Year','Number of Spouts','Latitude', 'Longitude']]
dfs_e = [hail_df,rain_df,tornado_df,lightning_df,wind_df,waterspout_df]

In [25]:
df2 = pd.DataFrame({'Year':[int(i) for i in range(1900,2021)]})
new_df = lightning_df.groupby('Year').count().iloc[:,0].reset_index()
new_df['Year'] = new_df['Year'].astype(int)
new_df2 = hail_df.groupby('Year').count().iloc[:,0].reset_index()
new_df2['Year'] = new_df2['Year'].astype(int)
new_df3 = rain_df.groupby('Year').mean().iloc[:,0].reset_index()
new_df3['Year'] = new_df3['Year'].astype(int)
new_df4 = tornado_df.groupby('Year').count().iloc[:,0].reset_index()
new_df4['Year'] = new_df4['Year'].astype(int)
new_df5 = waterspout_df.groupby('Year').count().iloc[:,0].reset_index()
new_df5['Year'] = new_df5['Year'].astype(int)
new_df6 = wind_df.groupby('Year').mean().iloc[:,0].reset_index()
new_df6['Year'] = new_df6['Year'].astype(int)
df2 = df2.merge(new_df,on="Year",how="left")
df2 = df2.merge(new_df2,on="Year",how="left")
df2 = df2.merge(new_df3,on="Year",how="left")
df2 = df2.merge(new_df4,on="Year",how="left")
df2 = df2.merge(new_df5,on="Year",how="left")
df2 = df2.merge(new_df6,on="Year",how="left")
df2 = df2.fillna(0)
summary_df = df2.rename({'Hail size':"hails_count", 'Intense precipitation amount': "rain_avg", 'Max speed (km/h)':"tornado_count",
       'Flash rate':'lightning_count', 'Max Mean Wind speed':"wind_speed_avg", 'Number of Spouts':'waterspouts_count'},axis=1)
summary_df.head()

,Year,lightning_count,hails_count,rain_avg,tornado_count,waterspouts_count,wind_speed_avg
0,1900,0.0,5.0,31.400000,3.0,0.0,0.0
1,1901,1.0,7.0,81.600000,7.0,0.0,0.0
2,1902,0.0,11.0,81.666667,7.0,0.0,0.0
3,1903,0.0,10.0,60.842105,9.0,0.0,0.0
4,1904,1.0,5.0,60.375000,6.0,0.0,0.0


In [26]:
ff_df = final_df.merge(summary_df,on="Year",how="left")
ff_df['Year'] = ff_df['Year'].astype(int)
ff_df.fillna(0,inplace=True)

In [27]:
final_df = ff_df[['Year','anomalie','max_temp', 'min_temp', 
'rainfall_anom','rainfall','day_cloud', 'drough_index', 'lightning_count', 'hails_count',
       'rain_avg', 'tornado_count', 'waterspouts_count', 'wind_speed_avg']].\
           rename({'anomalie':'avg_temp'},axis=1)

In [28]:
final_df = final_df[final_df['Year']>= 1910]

In [29]:
sea_level = pd.read_csv('cleaned/sea_level_global.csv',index_col='Year',parse_dates=True)
sea_level.reset_index(inplace=True)
sea_level['Year'] = pd.to_datetime(sea_level['Year']).dt.year
sea_level = sea_level[['Year','diff','rise_year']]
sea_level.head()

,Year,diff,rise_year
0,1992,0.00,0.00
1,1993,7.46,7.46
2,1994,13.66,6.20
3,1995,11.86,-1.80
4,1996,10.24,-1.62


In [30]:
final_df = final_df.merge(sea_level,on='Year',how='left')
final_df.fillna(0,inplace=True)

In [31]:
final_df.to_csv('cleaned/australia_extreme_summary.csv',index=False)
